---
title: "A comparison of the NCRPG with the CPPA on the Grassmann manifold"
author: "Hajg Jasa"
date: 05/09/2025
# engine: julia
---


## Introduction

In this example we compare the Nonconvex Riemannian Proximal Gradient (NCRPG) method [BergmannJasaJohnPfeffer:2025:1](@cite) with the Cyclic Proximal Point Algorithm, which was introduced in [Bacak:2014](@cite), on the space of symmetric positive definite matrices and on hyperbolic space.
This example reproduces the results from [BergmannJasaJohnPfeffer:2025:1](@cite), Section 5.4.


In [ ]:
#| echo: false
#| code-fold: true
#| output: false
using Pkg;
cd(@__DIR__)
Pkg.activate("."); # for reproducibility use the local tutorial environment.

Pkg.develop(path="../") # a trick to work on the local dev version

export_orig = true
export_table = true
export_result = true
benchmarking = true

experiment_name = ""
results_folder = joinpath(@__DIR__, experiment_name)
!isdir(results_folder) && mkdir(results_folder)

In [ ]:
#| output: false
using PrettyTables
using BenchmarkTools
using CSV, DataFrames
using ColorSchemes, Plots
using Random, LinearAlgebra, LRUCache
using ManifoldDiff, Manifolds, Manopt, ManoptExamples

## The Problem

Let $\mathcal M$ be a Riemannian manifold and $\{q_1,\ldots,q_N\} \in \mathcal M$ denote $N = 1000$
Gaussian random data points.
Let $g \colon \mathcal M \to \mathbb R$ be defined by

```math
g(p) = \sum_{j = 1}^N w_j \, \mathrm{dist}(p, q_j)^2,
```

where $w_j$, $j = 1, \ldots, N$ are positive weights such that $\sum_{j = 1}^N w_j = 1$.
In our experiments, we choose the weights $w_j = \frac{1}{2N}$.
Observe that the function $g$ is strongly convex with respect to the Riemannian metric on $\mathcal M$.
The Riemannian geometric median $p^*$ of the dataset $\{q_1,\ldots,q_N\}$

```math
\mathcal D = \{
    q_1,\ldots,q_N \, \vert \, q_j \in \mathcal M\text{ for all } j = 1,\ldots,N
\}
```

is then defined as

```math
    p^* \coloneqq \operatorname*{arg\,min}_{p \in \mathcal M} g(p),
```

where equality is justified since $p^*$ is uniquely determined on Hadamard manifolds. 

Let now $\bar q \in \cM$ be a given point, and let $h \colon \mathcal M \to \mathbb R$ be defined by

```math
h(p) = \alpha \mathrm{dist}(p, \bar q).
``` 

We define our total objective function as $f = g + h$.
Notice that this objective function is strongly convex with respect to the Riemannian metric on $\mathcal M$ thanks to $g$.
The goal is to find the minimizer of $f$ on $\mathcal M$, which heuristically is an interpolation between the geometric median $p^*$ and $\bar q$.


## Numerical Experiment

We initialize the experiment parameters, as well as some utility functions.

In [ ]:
#| output: false
random_seed = 100
experiment_name = "NCRPG-Grassmann"
results_folder = joinpath(@__DIR__, experiment_name)
!isdir(results_folder) && mkdir(results_folder)

atol = 1e-8
max_iters = 5000
N = 1000 # number of data points
α = 3/2 # weight for the median component (h)
gr_dims = [(5, 2), (10, 4), (50, 10), (100, 20), (200, 40)] # dimensions of the Grassmann manifold

In [ ]:
#| output: false
# Objective, gradient, and proxes
g(M, p, data) = 1/2length(data) * sum(distance.(Ref(M), data, Ref(p)).^2)
grad_g(M, p, data) = 1/length(data) * sum(ManifoldDiff.grad_distance.(Ref(M), data, Ref(p), 2))
# 
h(M, p, q) = α * distance(M, p, q)
prox_h(M, λ, p, q) = ManifoldDiff.prox_distance(M, α * λ, q, p, 1)
# 
f(M, p, data, q) = g(M, p, data) + h(M, p, q)
# CPPA needs the proximal operators for the total objective
function proxes_f(data, q)
    proxes = Function[(M, λ, p) -> ManifoldDiff.prox_distance(M, λ / length(data), di, p, 2) for di in data]
    push!(proxes, (M, λ, p) -> ManifoldDiff.prox_distance(M, α * λ, q, p, 1))
    return proxes
end
# Function to generate points close to the given point p
function close_point(M, p, tol; retraction_method=Manifolds.default_retraction_method(M, typeof(p)))
    X = rand(M; vector_at = p)
    X .= tol * rand() * X / norm(M, p, X)
    return retract(M, p, X, retraction_method)
end
# Estimate Lipschitz constant of the gradient of g
function estimate_lipschitz_constant(M, g, grad_g, anchor, R, N=1000)
    constants = []
    for i in 1:N
        p = close_point(M, anchor, R)
        q = close_point(M, anchor, R)

        push!(constants, 2/distance(M, q, p)^2 * (g(M, q) - g(M, p) - inner(M, p, grad_g(M, p), log(M, p, q))))
    end
    return maximum(constants)
end

We introduce some keyword arguments for the solvers we will use in this experiment

In [ ]:
#| output: false
pgm_kwargs(initial_stepsize) = [
    :record => [:Iteration, :Cost, :Iterate],
    :return_state => true,
    :stepsize => ProxGradBacktracking(; strategy=:nonconvex, initial_stepsize=initial_stepsize),
    :stopping_criterion => StopWhenAny(
        StopWhenGradientMappingNormLess(atol), StopAfterIteration(max_iters)
    ),
]
pgm_bm_kwargs(initial_stepsize) = [
    :record => [:Iteration, :Cost, :Iterate],
    :return_state => true,
    :stepsize => ProxGradBacktracking(; strategy=:nonconvex,   
        initial_stepsize=initial_stepsize),
    :stopping_criterion => StopWhenAny(
        StopWhenGradientMappingNormLess(atol), StopAfterIteration(max_iters)
    ), 
]

cppa_kwargs(M) = [
    :record => [:Iteration, :Cost, :Iterate],
    :return_state => true,
    :stopping_criterion => StopWhenAny(
        StopAfterIteration(max_iters), StopWhenChangeLess(M, atol)
    ),
]
cppa_bm_kwargs(M) = [
    :record => [:Iteration, :Cost, :Iterate],
    :return_state => true,
    :stopping_criterion => StopWhenAny(
        StopAfterIteration(max_iters), StopWhenChangeLess(M, atol)
    ),
]

Before running the experiments, we initialize data collection functions that we will use later

In [ ]:
#| output: false
global col_names_1 = [
    :Dimension,
    :Iterations_1,
    :Time_1,
    :Objective_1,
    :Iterations_2,
    :Time_2,
    :Objective_2,
]
col_types_1 = [
    Int64,
    Int64,
    Float64,
    Float64,
    Float64,
    Float64,
    Float64,
]
named_tuple_1 = (; zip(col_names_1, type[] for type in col_types_1 )...)
global col_names_2 = [
    :Dimension,
    :Iterations,
    :Time,
    :Objective,
]
col_types_2 = [
    Int64,
    Int64,
    Float64,
    Float64,
]
named_tuple_2 = (; zip(col_names_2, type[] for type in col_types_2 )...)
function initialize_dataframes(results_folder, experiment_name, subexperiment_name, named_tuple_1, named_tuple_2)
    A1 = DataFrame(named_tuple_1)
    CSV.write(
        joinpath(
            results_folder,
            experiment_name * 
            # "_$subexperiment_name" * 
            "-Comparisons.csv",
        ),
        A1;
        header=false,
    )
    # A2 = DataFrame(named_tuple_2)
    # CSV.write(
    #     joinpath(
    #         results_folder,
    #         experiment_name * "_$subexperiment_name" * "-Comparisons-Subgrad.csv",
    #     ),
    #     A2;
    #     header=false,
    # )
    return A1#, A2
end

In [ ]:
#| output: false
function export_dataframes(M, records, times, results_folder, experiment_name, subexperiment_name, col_names_1, col_names_2)
    B1 = DataFrame(;
        Dimension=manifold_dimension(M),
        Iterations_1=maximum(first.(records[1])),
        Time_1=times[1],
        Objective_1=minimum([r[2] for r in records[1]]),
        Iterations_2=maximum(first.(records[2])),
        Time_2=times[2],
        Objective_2=minimum([r[2] for r in records[2]]),
    )
    # B2 = DataFrame(;
    #     Dimension=manifold_dimension(M),
    #     Iterations=median(last.(records[3])),
    #     Time=times[3],
    #     Objective=minimum([r[2] for r in records[3]]),
    # )
    return B1#, B2
end
function write_dataframes(
    B1, 
    # B2, 
    results_folder, 
    experiment_name, 
    subexperiment_name
)
    CSV.write(
        joinpath(
            results_folder,
            experiment_name *
            # "_$subexperiment_name" *
            "-Comparisons.csv",
            # -Convex-Prox.csv",
        ),
        B1;
        append=true,
    )
    # CSV.write(
    #     joinpath(
    #         results_folder,
    #         experiment_name *
    #         "_$subexperiment_name" *
    #         "-Comparisons-Subgrad.csv",
    #     ),
    #     B2;
    #     append=true,
    # )
end

In [ ]:
#| output: false
subexperiment_name = "Gr"
k_max_gr = 2.0
global A1_Gr = initialize_dataframes(
    results_folder,
    experiment_name,
    subexperiment_name,
    named_tuple_1,
    named_tuple_2
)

for (n, m) in gr_dims

    Random.seed!(random_seed)

    M = Grassmann(Int(n), Int(m))
    data = [rand(M) for _ in 1:N]
    q = rand(M) # we can artificially craft a point for the median component, i.e. h
    p0 = rand(M) #data[minimum(Tuple(findmax(dists)[2]))]

    g_gr(M, p) = g(M, p, data)
    # h_gr(M, p) = h(M, p, q)
    grad_g_gr(M, p) = grad_g(M, p, data)
    proxes_f_gr = proxes_f(data, q)
    prox_h_gr(M, λ, p) = prox_h(M, λ, p, q)
    f_gr(M, p) = f(M, p, data, q)

    # D = 5/2 * maximum([distance(M, p0, di) for di in vcat(data, [q])])
    # L_g = 2 * estimate_lipschitz_constant(M, g_gr, grad_g_gr, p0, D)
    initial_stepsize = 1.0 #3/(2 * L_g)

    # Optimization
    pgm = proximal_gradient_method(M, f_gr, g_gr, grad_g_gr, prox_h_gr, p0; pgm_kwargs(initial_stepsize)...)
    pgm_result = get_solver_result(pgm)
    pgm_record = get_record(pgm)

    cppa = cyclic_proximal_point(M, f_gr, proxes_f_gr, p0; cppa_kwargs(M)...)
    cppa_result = get_solver_result(cppa)
    cppa_record = get_record(cppa)

    records = [
        pgm_record,
        cppa_record,
    ]

    if benchmarking
        pgm_bm = @benchmark proximal_gradient_method($M, $f_gr, $g_gr, $grad_g_gr, $prox_h_gr, $p0; $pgm_bm_kwargs($initial_stepsize)...)
        cppa_bm = @benchmark cyclic_proximal_point($M, $f_gr, $proxes_f_gr, $p0; cppa_bm_kwargs($M)...)
        
        times = [
            median(pgm_bm).time * 1e-9,
            median(cppa_bm).time * 1e-9,
        ]

        B1 = export_dataframes(
            M,
            records,
            times,
            results_folder,
            experiment_name,
            subexperiment_name,
            col_names_1,
            col_names_2,
        )

        append!(A1_Gr, B1)
        # append!(A2_Gr, B2)
        (export_table) && (write_dataframes(B1, results_folder, experiment_name, subexperiment_name))
    end
end

We can take a look at how the algorithms compare to each other in their performance with the following table, where columns 2 to 4 relate to the CRPG, while columns 5 to 7 refer to the CPPA...

In [ ]:
#| echo: false
#| code-fold: true
benchmarking && pretty_table(A1_Gr, tf = tf_markdown, header=col_names_1)

## Technical details

This tutorial is cached. It was last run on the following package versions.


In [ ]:
#| code-fold: true
using Pkg
Pkg.status()

In [ ]:
#| code-fold: true
#| echo: false
#| output: asis
using Dates
println("This tutorial was last rendered $(Dates.format(now(), "U d, Y, H:M:S")).");

## Literature


````{=commonmark}
```@bibliography
Pages = ["NCRPG-Grassmann.md"]
Canonical=false
```
````